# Рекомендация тарифов для оператора мобильной связи

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Оператор хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Цель - построить модель для задачи классификации, которая выберет подходящий тариф.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Оглавление

1. Первичный анализ данных
2. Подготовка данных для работы с моделями. Выборки, разбиение
3. Исследование моделей машинного обучения
4. Проверка лучшей модели на тестовой выборке
5. Проверка лучшей модели на адекватность

In [2]:
# Библиотеки которые будут использованы в данном проекте.
import pandas as pd
import math
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# Первичный анализ данных

In [2]:
try:
    df = pd.read_csv('users_behavior.csv')
except FileNotFoundError: 
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
df['calls'] = df['calls'].astype('int64')
df['messages'] = df['messages'].astype('int64')

In [6]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

## Вывод первичного анализа поступивших данных
Согласно первичному анализу данные не нуждаются в преобразованиях. Данные изначально валидны и не содержат пропусков.
- Были изменены типы данных в столбцах `calls`, `messages` с float64 -> int64
- Первичное исследование показало, что в целевом признаке есть явный дисбаланс в сторону класса - 0. Учтем данный факт при разбиении датасета на выборки.

# Подготовка данных для работы с моделями. Выборки, разбиение

In [7]:
# Разделим датасет на две части. Первая содержит все признаки. Вторая - целевой признак.
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

In [8]:
random_state = 12345

In [9]:
#Сначала отделим от датасета 60% наблюдений - это будет обучающая выборка. 
#Оставщиеся данные поделим пополам - на тестовую и валидационную выборки.
#Таким образом мы разобьем данные в соотношении 60/20/20.
train_features, valid_features, train_target, valid_target = train_test_split(X, y, test_size=0.4, random_state=random_state, stratify=y)
valid_features, test_features, valid_target, test_target = train_test_split(valid_features, valid_target, test_size=0.5, random_state=random_state, stratify=valid_target)


## Вывод по разбиению данных
Данные были разделены на три выборки в соотношении 60\20\20 соответсвенно:

- `train_features` -  тренировочный набор данных без целевого признака
- `train_target` - тренировочный набор данных с целевым признаком


- `valid_features` - валидационный набор данных без целевого признака
- `valid_target` - валидационный набор данных с целевым признаком


- `test_features` - тестовый набор данных без целевого признака
- `test_target` - тестовый набор данных с целевым признаком

# Исследование моделей машинного обучения

In [10]:
# Нам предстоит проделать большое количество операций по оценке эффективности моделей при разных значениях гиперпареметров
# Создадим функцию благодаря которой сможем наглядно увидеть какая модель и при каких параметрах окажется лучшей
best_accuracy = dict()

def best_accuracy_func(model, model_info, accuracy):
    best_accuracy['model'] = model
    best_accuracy['model_info'] = model_info
    best_accuracy['accuracy'] = accuracy
    return best_accuracy

# Значения по умолчанию
best_accuracy = best_accuracy_func('Undefined', 'Undefined', 0)

In [11]:
# Посмотрим на Дерево решений
for depth in range(1, 11):
    modelDecisionTree = DecisionTreeClassifier(random_state=random_state, max_depth=depth) 
    modelDecisionTree.fit(train_features,train_target)
    predictions_valid = modelDecisionTree.predict(valid_features)
    
    accuracy = accuracy_score(valid_target, predictions_valid)
    
    model_info = 'Descision Tree; max_depth= ' + str(depth)
    
    if (accuracy > best_accuracy['accuracy']):
        best_accuracy_dict = best_accuracy_func(modelDecisionTree, model_info, accuracy)
    print(model_info,'; accuracy =', accuracy)

Descision Tree; max_depth= 1 ; accuracy = 0.7402799377916018
Descision Tree; max_depth= 2 ; accuracy = 0.7729393468118196
Descision Tree; max_depth= 3 ; accuracy = 0.7776049766718507
Descision Tree; max_depth= 4 ; accuracy = 0.7542768273716952
Descision Tree; max_depth= 5 ; accuracy = 0.7853810264385692
Descision Tree; max_depth= 6 ; accuracy = 0.7744945567651633
Descision Tree; max_depth= 7 ; accuracy = 0.7869362363919129
Descision Tree; max_depth= 8 ; accuracy = 0.80248833592535
Descision Tree; max_depth= 9 ; accuracy = 0.7822706065318819
Descision Tree; max_depth= 10 ; accuracy = 0.7729393468118196


In [12]:
# Посмотрим на Случайный лес
for est in range(10, 51, 5):
    for depth in range (1, 11):
        modelRandomForest = RandomForestClassifier(n_estimators=est, random_state=random_state, max_depth=depth)
        modelRandomForest.fit(train_features, train_target)
        predictions_valid = modelRandomForest.predict(valid_features)
        
        accuracy = accuracy_score(valid_target, predictions_valid)
        
        model_info = "Random Forest; n_estimators = " + str(est) + " ; max_depth= " + str(depth)
        if (accuracy > best_accuracy['accuracy']):
            best_accuracy_dict = best_accuracy_func(modelRandomForest, model_info, accuracy)
        print(model_info,'; accuracy =', accuracy)
    print() 

Random Forest; n_estimators = 10 ; max_depth= 1 ; accuracy = 0.7573872472783826
Random Forest; n_estimators = 10 ; max_depth= 2 ; accuracy = 0.776049766718507
Random Forest; n_estimators = 10 ; max_depth= 3 ; accuracy = 0.7947122861586314
Random Forest; n_estimators = 10 ; max_depth= 4 ; accuracy = 0.7916018662519441
Random Forest; n_estimators = 10 ; max_depth= 5 ; accuracy = 0.7884914463452566
Random Forest; n_estimators = 10 ; max_depth= 6 ; accuracy = 0.8040435458786936
Random Forest; n_estimators = 10 ; max_depth= 7 ; accuracy = 0.7931570762052877
Random Forest; n_estimators = 10 ; max_depth= 8 ; accuracy = 0.7978227060653188
Random Forest; n_estimators = 10 ; max_depth= 9 ; accuracy = 0.8040435458786936
Random Forest; n_estimators = 10 ; max_depth= 10 ; accuracy = 0.7993779160186625

Random Forest; n_estimators = 15 ; max_depth= 1 ; accuracy = 0.7558320373250389
Random Forest; n_estimators = 15 ; max_depth= 2 ; accuracy = 0.7869362363919129
Random Forest; n_estimators = 15 ; max_

In [13]:
# Посмотрим на Логическую регрессию 
modelLogisticRegression = LogisticRegression(random_state=random_state)
modelLogisticRegression.fit(train_features, train_target)
predictions_valid = modelLogisticRegression.predict(valid_features)

accuracy = accuracy_score(valid_target, predictions_valid)
        
model_info = 'Logistic Regression'
if (accuracy > best_accuracy_dict['accuracy']):
    best_accuracy_dict = best_accuracy_func(modelLogisticRegression, model_info, accuracy)
print(model_info,'; accuracy =', accuracy)

Logistic Regression ; accuracy = 0.7387247278382582


In [14]:
# Посмотрим на Линейную регрессию
modelLinearRegression = LinearRegression()
modelLinearRegression.fit(train_features, train_target)
predictions_valid = modelLinearRegression.predict(valid_features)

accuracy = math.sqrt(mean_squared_error(valid_target, predictions_valid))
        
model_info = 'Linear Regression'
if (accuracy > best_accuracy_dict['accuracy']):
    best_accuracy_dict = best_accuracy_func(modelLinearRegression, model_info, accuracy)
print(model_info,'; accuracy =', accuracy)

Linear Regression ; accuracy = 0.4439015580302285


In [15]:
# Посмотрим, какая модель оказалась лучше всего
best_accuracy_dict

{'model': RandomForestClassifier(max_depth=9, n_estimators=40, random_state=12345),
 'model_info': 'Random Forest; n_estimators = 40 ; max_depth= 9',
 'accuracy': 0.8211508553654744}

## Вывод по исследованию моделей машинного обучения
- Лучший результат в точности показала модель Случайного леса с параметрами max_depth=9, n_estimators=40 выдав показатель точности в 0.82
- Худший результат в точности показала модель Линейной регрессии, выдав показатель точности в 0.44
- Решающее дерево показало достойные результаты с точностью в районе 0.77


## Проверка лучшей модели на тестовой выборке

In [16]:
# Настало время проверить нашу модель на тестовых выборках
model = best_accuracy['model']
test_predicted = model.predict(test_features)
print('Test accuracy =', accuracy_score(test_target, test_predicted))

Test accuracy = 0.8087091757387247


## Проверка лучшей модели на адекватность

In [19]:
random_predictions = pd.Series(test_target.apply(lambda x: round(random.random())), index=test_target.index)

# оценим точность на этих данных
print('Random predictions accuracy = ', accuracy_score(test_target, random_predictions))

Random predictions accuracy =  0.52099533437014


Модель прошла проверку на адекватность

# Итоговый вывод

- Были изучены предложение данные.
- Полученный датасет был разделен на три выборки в пропорции 60\20\20.
- Исследованы четыре модели машинного обучения с различными гиперпараметрами.
- Определена лучшая модель машинного обучения с перечнем параметров для неё. (Случайный лес с параметрами max_depth=8, n_estimators=30 выдал показатель точности в 0.83)
- Лучшая модель машинного обучения прошла проверку на адекватность.
- Работа лучшей модели на тестовой выборке показало точность в 0.808
